In [1]:
# IMPORT PACKAGES

import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', None)
import os
import glob
import re
import shutil
import warnings
import matplotlib.pyplot as plt
import math

In [2]:
#-------------------------------------------------------------------------------------------------------------
# DEFINE INPUTS
#-------------------------------------------------------------------------------------------------------------
dataset = "D7" # name of dataset which was used for the inversion
model = "reyk_final_model" # name of inversion
#-------------------------------------------------------------------------------------------------------------

depth_slice = True
EW_slice = True
NS_slice = True

depth = "10" # depth slice depth in km
EW = "63.90" # EW slice latitude in degrees
NS = "21.90" # NS slice longitude in degrees
slice_par = [depth, EW, NS]

#--------------------------------------------------------------------------------------------------------------

depth_dist = 1.0 # max. distance of events from depth slice location to be plotted onto the figure in km
#ew_dist = 0.0135 # max. distance of events from EW slice location to be plotted onto the figure in degrees
ew_dist = 0.0090 # 1 km
#ns_dist = 0.0306 # max. distance of events from NS slice location to be plotted onto the figure in degrees
ns_dist = 0.0204 # 1 km
evdist = [depth_dist, ew_dist, ns_dist]

#--------------------------------------------------------------------------------------------------------------

gmt_files_path = "/raid4/hra35/gmt_figures/model_slice_scripts/event_subsets"


path_to_models = "/home/hra35/Documents/events/residuals"
path_to_model_files = os.path.join(path_to_models, model)

p_receivers = os.path.join(path_to_model_files, "receivers_p.in")
s_receivers = os.path.join(path_to_model_files, "receivers_s.in")


In [3]:
# FUNCTION FOR EXTRACTING EVENT FILES

def plot_subset_of_events(gmt_files_path, depth_slice, EW_slice, NS_slice, slice_par, evdist, p_receivers, s_receivers):
        
    # read the receivers.in files to get event coordinates
    with open(p_receivers, 'r') as file:
        p_receivers_lines = file.readlines()
        
    # read the receivers.in files to get event coordinates
    with open(s_receivers, 'r') as file:
        s_receivers_lines = file.readlines()

    # save number of P picks
    p_count = int(p_receivers_lines[0].split()[0])
    #print(p_count)
    
    # save event coordinates into dataframe
    p_df = pd.DataFrame(data=None, columns=["lat","lon","depth"])
    
    for i in range(p_count):
        
        event_line = p_receivers_lines[1+4*i]
        entries = event_line.split()
        dep = entries[0]
        lat = entries[1]
        lon = entries[2]
    
        data_row = {"lat": lat, "lon": lon, "depth": dep}
        p_df = p_df.append(data_row, ignore_index=True)
                        

    # save number of S picks
    s_count = int(s_receivers_lines[0].split()[0])
    #print(s_count)
    
    # save event coordinates into dataframe
    s_df = pd.DataFrame(data=None, columns=["lat","lon","depth"])
    
    for i in range(s_count):
        
        event_line = s_receivers_lines[1+4*i]
        entries = event_line.split()
        dep = entries[0]
        lat = entries[1]
        lon = entries[2]
    
        data_row = {"lat": lat, "lon": lon, "depth": dep}
        s_df = s_df.append(data_row, ignore_index=True)
    
    #print(s_df)
    
    # concatenate the two dataframes and remove duplicates
    all_df = pd.concat([p_df, s_df], ignore_index=True)
    
    numerical_columns = ['lat', 'lon', 'depth']
    for column in numerical_columns:
        all_df[column] = all_df[column].astype(float)
        
    #print("Number of picks is ", len(all_df))
    all_df.drop_duplicates(inplace=True)
    #print("Number of events is ", len(all_df))
    
    # output requested files

    evdist_dp = float(evdist[0])
    evdist_ew = float(evdist[1])
    evdist_ns = float(evdist[2])
    
    depthf = float(slice_par[0])
    EWf = float(slice_par[1])
    NSf = 360-float(slice_par[2])
    
    if depth_slice == True:
        mask = ((all_df["depth"] <= (depthf + evdist_dp)) & (all_df["depth"] >= (depthf - evdist_dp)))
        depth_df = all_df[mask]
        depth_df[['lon', 'lat']].to_csv(os.path.join(gmt_files_path, f"{depth}_depth.xy"), sep='\t', float_format='%.6f', header=False, index=False)

    if EW_slice == True:
        mask = ((all_df["lat"] <= (EWf + evdist_ew)) & (all_df["lat"] >= (EWf - evdist_ew)))
        ew_df = all_df[mask]
        ew_df["depth"] = ew_df["depth"] * (-1)
        ew_df[['lon', 'depth']].to_csv(os.path.join(gmt_files_path, f"{EW}_EW.xy"), sep='\t', float_format='%.6f', header=False, index=False)
   
    if NS_slice == True:
        mask = ((all_df["lon"] <= (NSf + evdist_ns)) & (all_df["lon"] >= (NSf - evdist_ns)))
        ns_df = all_df[mask]
        ns_df["depth"] = ns_df["depth"] * (-1)
        ns_df[['lat', 'depth']].to_csv(os.path.join(gmt_files_path, f"{NS}_NS.xy"), sep='\t', float_format='%.6f', header=False, index=False)
    
    return

In [4]:
ew_slices = ["63.84", "63.85", "63.86", "63.87", "63.88", "63.89", "63.90", "63.91", "63.92", "63.93"]
ns_slices = ["21.95", "22.00", "22.05", "22.10", "22.15", "22.20", "22.25", "22.30", "22.35", "22.40", "22.45", "22.50", "22.55"]
depth_slices = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15"]

depth_slice = False
EW_slice = True
NS_slice = False

for ew_slice in ew_slices:
    EW = ew_slice
    slice_par = [depth, EW, NS]
    plot_subset_of_events(gmt_files_path, depth_slice, EW_slice, NS_slice, slice_par, evdist, p_receivers, s_receivers)
    
depth_slice = False
EW_slice = False
NS_slice = True

for ns_slice in ns_slices:
    NS = ns_slice
    slice_par = [depth, EW, NS]
    plot_subset_of_events(gmt_files_path, depth_slice, EW_slice, NS_slice, slice_par, evdist, p_receivers, s_receivers)
    
depth_slice = True
EW_slice = False
NS_slice = False

for d_slice in depth_slices:
    depth = d_slice
    slice_par = [depth, EW, NS]
    plot_subset_of_events(gmt_files_path, depth_slice, EW_slice, NS_slice, slice_par, evdist, p_receivers, s_receivers)

14165
13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337.666046  3.83333325
13351  63.8420753  337.592712  3.50000000
13352  63.8730545  337.541595  4.66666651

[13353 rows x 3 columns]
Number of picks is  27518
Number of events is  1284
14165


/tmp/ipykernel_1690144/2001228187.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ew_df["depth"] = ew_df["depth"] * (-1)


13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337.666046  3.83333325
13351  63.8420753  337.592712  3.50000000
13352  63.8730545  337.541595  4.66666651

[13353 rows x 3 columns]
Number of picks is  27518
Number of events is  1284
14165


/tmp/ipykernel_1690144/2001228187.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ew_df["depth"] = ew_df["depth"] * (-1)


13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337.666046  3.83333325
13351  63.8420753  337.592712  3.50000000
13352  63.8730545  337.541595  4.66666651

[13353 rows x 3 columns]
Number of picks is  27518
Number of events is  1284
14165


...

/tmp/ipykernel_1690144/2001228187.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ns_df["depth"] = ns_df["depth"] * (-1)


13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337.666046  3.83333325
13351  63.8420753  337.592712  3.50000000
13352  63.8730545  337.541595  4.66666651

[13353 rows x 3 columns]
Number of picks is  27518
Number of events is  1284
14165


/tmp/ipykernel_1690144/2001228187.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ns_df["depth"] = ns_df["depth"] * (-1)


13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337.666046  3.83333325
13351  63.8420753  337.592712  3.50000000
13352  63.8730545  337.541595  4.66666651

[13353 rows x 3 columns]
Number of picks is  27518
Number of events is  1284
14165
13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337

13353
              lat         lon       depth
0      63.8863945  337.753876  6.77777767
1      63.9249992  337.818329  6.05555534
2      63.9307213  337.833893  4.61111116
3      63.9230957  337.837219  5.77777767
4      63.9269066  337.839447  4.77777767
...           ...         ...         ...
13348  63.9102249  337.794983  7.55555534
13349  63.9097519  337.772766  3.83333325
13350  63.8759155  337.666046  3.83333325
13351  63.8420753  337.592712  3.50000000
13352  63.8730545  337.541595  4.66666651

[13353 rows x 3 columns]
Number of picks is  27518
Number of events is  1284
